In [7]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

In [8]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

useducation_df = pd.read_sql_query('select * from useducation',con=engine)
# No need for an open connection, 
# because you're only doing a single query
engine.dispose()

In [9]:
useducation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1492 non-null   object 
 1   STATE                         1492 non-null   object 
 2   YEAR                          1492 non-null   int64  
 3   ENROLL                        1229 non-null   float64
 4   TOTAL_REVENUE                 1280 non-null   float64
 5   FEDERAL_REVENUE               1280 non-null   float64
 6   STATE_REVENUE                 1280 non-null   float64
 7   LOCAL_REVENUE                 1280 non-null   float64
 8   TOTAL_EXPENDITURE             1280 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1280 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1280 non-null   float64
 11  OTHER_EXPENDITURE             1229 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1280 non-null   float64
 13  GRA

In [10]:
useducation_df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [13]:
# Finding the fraction of missing values for each variable
useducation_df.isnull().sum()*100/useducation_df.isnull().count()

PRIMARY_KEY                      0.000000
STATE                            0.000000
YEAR                             0.000000
ENROLL                          17.627346
TOTAL_REVENUE                   14.209115
FEDERAL_REVENUE                 14.209115
STATE_REVENUE                   14.209115
LOCAL_REVENUE                   14.209115
TOTAL_EXPENDITURE               14.209115
INSTRUCTION_EXPENDITURE         14.209115
SUPPORT_SERVICES_EXPENDITURE    14.209115
OTHER_EXPENDITURE               17.627346
CAPITAL_OUTLAY_EXPENDITURE      14.209115
GRADES_PK_G                     11.595174
GRADES_KG_G                      8.847185
GRADES_4_G                       8.780161
GRADES_8_G                       8.780161
GRADES_12_G                      8.780161
GRADES_1_8_G                     8.780161
GRADES_9_12_G                    8.780161
GRADES_ALL_G                    11.595174
AVG_MATH_4_SCORE                64.075067
AVG_MATH_8_SCORE                64.343164
AVG_READING_4_SCORE             64

In [16]:
# Choosing a strategy to deal with the missing values for each variable.
# Finding out for which variable would filling in the missing values with some value make sense
# For which might tossing out the records entirely make sense
useducation_df.PRIMARY_KEY.value_counts()

2008_VIRGINIA                3
2008_DISTRICT_OF_COLUMBIA    2
2009_DISTRICT_OF_COLUMBIA    2
2010_DISTRICT_OF_COLUMBIA    2
1992_ALABAMA                 1
                            ..
2001_MISSISSIPPI             1
2001_MINNESOTA               1
2001_MICHIGAN                1
2001_MASSACHUSETTS           1
2017_WYOMING                 1
Name: PRIMARY_KEY, Length: 1487, dtype: int64

In [17]:
useducation_df.STATE.value_counts()

DISTRICT_OF_COLUMBIA                        29
VIRGINIA                                    28
ALABAMA                                     26
PENNSYLVANIA                                26
NEW_HAMPSHIRE                               26
                                            ..
DOD_(OVERSEAS_AND_DOMESTIC_COMBINED)         1
U.S._VIRGIN_ISLANDS                          1
DEPARTMENT_OF_DEFENSE_EDUCATION_ACTIVITY     1
DEPARTMENT_OF_DEFENSE                        1
DOD                                          1
Name: STATE, Length: 80, dtype: int64

In [18]:
# For the columns 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', 'AVG_READING_4_SCORE','AVG_READING_8_SCORE' we should consider dropping the records entirely as the percentage of null values in these columns >50%.

In [20]:
# We should consider filling in the missing values for these columns 'ENROLL','TOTAL_REVENUE','FEDERAL_REVENUE','STATE_REVENUE','LOCAL_REVENUE','TOTAL_EXPENDITURE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE','GRADES_PK_G','GRADES_KG_G','GRADES_4_G','GRADES_8_G','GRADES_12_G','GRADES_1_8_G','GRADES_9_12_G','GRADES_ALL_G' as the percentage of missing values for these columns is low.

In [38]:
for i in ['ENROLL','TOTAL_REVENUE','FEDERAL_REVENUE','STATE_REVENUE','LOCAL_REVENUE','TOTAL_EXPENDITURE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE','GRADES_PK_G','GRADES_KG_G','GRADES_4_G','GRADES_8_G','GRADES_12_G','GRADES_1_8_G','GRADES_9_12_G','GRADES_ALL_G']:
    useducation_df[i] = useducation_df[i].fillna(useducation_df[i].mean())

In [39]:
useducation_df['ENROLL']
useducation_df['TOTAL_REVENUE']

0       2.678885e+06
1       1.049591e+06
2       3.258079e+06
3       1.711959e+06
4       2.626002e+07
            ...     
1487    9.092082e+06
1488    9.092082e+06
1489    9.092082e+06
1490    9.092082e+06
1491    9.092082e+06
Name: TOTAL_REVENUE, Length: 1492, dtype: float64

In [40]:
for i in ['ENROLL','TOTAL_REVENUE','FEDERAL_REVENUE','STATE_REVENUE','LOCAL_REVENUE','TOTAL_EXPENDITURE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE','GRADES_PK_G','GRADES_KG_G','GRADES_4_G','GRADES_8_G','GRADES_12_G','GRADES_1_8_G','GRADES_9_12_G','GRADES_ALL_G']:
    useducation_df[i] = useducation_df[i].fillna(useducation_df.groupby('YEAR')[i].mean())

In [41]:
useducation_df['TOTAL_REVENUE']

0       2.678885e+06
1       1.049591e+06
2       3.258079e+06
3       1.711959e+06
4       2.626002e+07
            ...     
1487    9.092082e+06
1488    9.092082e+06
1489    9.092082e+06
1490    9.092082e+06
1491    9.092082e+06
Name: TOTAL_REVENUE, Length: 1492, dtype: float64

In [42]:
# Filling in missing values using interpolation

for i in ['ENROLL','TOTAL_REVENUE','FEDERAL_REVENUE','STATE_REVENUE','LOCAL_REVENUE','TOTAL_EXPENDITURE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE','GRADES_PK_G','GRADES_KG_G','GRADES_4_G','GRADES_8_G','GRADES_12_G','GRADES_1_8_G','GRADES_9_12_G','GRADES_ALL_G']:
    useducation_df[i] = useducation_df[i].fillna(useducation_df[i].interpolate())

In [44]:
useducation_df['TOTAL_REVENUE']

0       2.678885e+06
1       1.049591e+06
2       3.258079e+06
3       1.711959e+06
4       2.626002e+07
            ...     
1487    9.092082e+06
1488    9.092082e+06
1489    9.092082e+06
1490    9.092082e+06
1491    9.092082e+06
Name: TOTAL_REVENUE, Length: 1492, dtype: float64

In [ ]:
# I couldn't find any meaningful difference in the results on using 3 methods of filling missing values.